<a href="https://colab.research.google.com/github/nokiddig/IT5-FTW/blob/main/Adversarial_Training_myModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Adversarial training**

## **Tải dataset và giải nén**

In [ ]:
!pip install -U --no-cache-dir gdown --pre

In [ ]:
#!gdown https://drive.google.com/uc?id=1c1xAis_x_8AGA_QuCo0BQ0GjADzzqz7C
!gdown https://drive.google.com/uc?id=1neDeCdJbyW0t-dqAMYpDFkcQq4eujhl7

Downloading...
From: https://drive.google.com/uc?id=1neDeCdJbyW0t-dqAMYpDFkcQq4eujhl7
To: /content/new_Train.zip
100% 104M/104M [00:00<00:00, 170MB/s] 


In [ ]:
!unzip new_Train.zip

Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
  inflating: new_Train/5/00005_00055_00004.png  
  inflating: new_Train/5/00005_00055_00005.png  
  inflating: new_Train/5/00005_00055_00006.png  
  inflating: new_Train/5/00005_00055_00007.png  
  inflating: new_Train/5/00005_00055_00008.png  
  inflating: new_Train/5/00005_00055_00009.png  
  inflating: new_Train/5/00005_00055_00010.png  
  inflating: new_Train/5/00005_00055_00011.png  
  inflating: new_Train/5/00005_00055_00012.png  
  inflating: new_Train/5/00005_00055_00013.png  
  inflating: new_Train/5/00005_00055_00014.png  
  inflating: new_Train/5/00005_00055_00015.png  
  inflating: new_Train/5/00005_00055_00016.png  
  inflating: new_Train/5/00005_00055_00017.png  
  inflating: new_Train/5/00005_00055_00018.png  
  inflating: new_Train/5/00005_00055_00019.png  
  inflating: new_Train/5/00005_00055_00020.png  
  inflating: new_Train/5/00005_00055_00021.png  
  inflating: new_Train/5/00005_00055_00022.png  
  inflating:

## **Importing necessary libraries**

In [ ]:
from tensorflow.keras import models, layers
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score

import numpy as np
import pandas as pd
import os
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib 
import seaborn as sns
import numpy
from PIL import Image

from keras.models import load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
import argparse

In [ ]:
print(tf.__version__)

2.8.0


## **Reading the Data**

In [ ]:
imgs_path = os.getcwd() + '/new_Train'
data_list = []
labels_list = []
classes_list = 12
for i in range(classes_list):
    i_path = os.path.join(imgs_path, str(i)) #0-11
    for img in os.listdir(i_path):
        im = Image.open(i_path +'/'+ img)
        im = im.resize((32,32))
        im = np.array(im)
        data_list.append(im)
        labels_list.append(i)
data = np.array(data_list)
labels = np.array(labels_list)

In [ ]:

# data_list = []
# labels_list = []

# im = Image.open('stop2.png')
# im = im.resize((32,32))
# im = np.array(im)
# data_list.append(im)
# labels_list.append(11)

# im = Image.open('stop2.png')
# im = im.resize((32,32))
# im = np.array(im)
# data_list.append(im)
# labels_list.append(11)

# data = np.array(data_list)
# labels = np.array(labels_list)
# #labels = to_categorical(labels)

## **Preparing the Dataset**

In [ ]:
!pip install keras-efficientnet

In [ ]:
def prep_dataset(X,y):
    X_prep = X.astype('float32')
    y_prep = to_categorical(np.array(y))
    X_prep = tf.keras.applications.mobilenet_v2.preprocess_input(X_prep)
    return (X_prep, y_prep)

X, y = prep_dataset(data,labels)

## **Splitting the dataset**

In [ ]:
X_train, X_val, Y_train, Y_val = train_test_split(data,labels, test_size=0.2, shuffle=True,stratify=labels)
X_val, X_test, Y_val, Y_test = train_test_split(X_val,Y_val, test_size=0.5, shuffle=True)

In [ ]:
!pip install -U --no-cache-dir gdown --pre

In [ ]:
# model mới 12class trên thang [-1; 1]
!gdown https://drive.google.com/uc?id=1j3NPgHWy0_lkUgR7XDcTXVjcA42iTy7T

# model 43 class
#!gdown https://drive.google.com/uc?id=1h6VtgnmjaCfteXnc5gcP5zvykvBjOPYV

Downloading...
From: https://drive.google.com/uc?id=1j3NPgHWy0_lkUgR7XDcTXVjcA42iTy7T
To: /content/my_model_12class.h5
100% 669M/669M [00:09<00:00, 67.3MB/s]


In [ ]:
model = load_model('my_model_12class.h5')
#model = load_model('my_model.h5')

## **Creating an adversarial example**

In [ ]:
# define the epsilon and learning rate constants
EPS = 20 / 255.0
LR = 0.1

optimizer = Adam(learning_rate=LR)
sccLoss = SparseCategoricalCrossentropy()

def clip_eps(tensor, eps):
	# clip the values of the tensor to a given range and return it
	return tf.clip_by_value(tensor, clip_value_min=-eps,
		clip_value_max=eps)

def generate_adversaries(model, baseImage, delta, classIdx, steps=100):
	# iterate over the number of steps
	for step in range(0, steps):
		# record our gradients
		with tf.GradientTape() as tape:
			# explicitly indicate that our perturbation vector should 
      # be tracked for gradient updates
			tape.watch(delta) 

      # add our perturbation vector to the base image and
			# preprocess the resulting image
			# baseImage = baseImage*0.5 + 0.5
			adversary = preprocess_input(baseImage + delta)
			# run this newly constructed image tensor through our
			# model and calculate the loss with respect to the
			# *original* class index
			predictions = model(adversary, training=False)
			loss = -sccLoss(tf.convert_to_tensor([classIdx]), predictions)
			
      # check to see if we are logging the loss value, and if
			# so, display it to our terminal
			if step % 5 == 0:
				print("step: {}, loss: {}...".format(step, loss.numpy()))
		  
    # calculate the gradients of loss with respect to the
		# perturbation vector
		gradients = tape.gradient(loss, delta)
	
  	# update the weights, clip the perturbation vector, and
		# update its value
		optimizer.apply_gradients([(gradients, delta)])
		delta.assign_add(clip_eps(delta, eps=EPS))

	# return the perturbation vector
	return delta

In [ ]:
def preprocess(image):
  image = Image.open(image)
  image = image.resize((32,32))
  image = numpy.expand_dims(image, axis=0)
  #image = numpy.array(image)
  image = numpy.array(image).astype(numpy.float32)
  image = tf.keras.applications.mobilenet_v2.preprocess_input(image)
  image = tf.convert_to_tensor(image)
  return image

In [ ]:
def generate_adversarial_batch(model, total, images, labels):
  while True:
    #initialize our perturbed images and labels
    perturbImages = []
    perturbLabels = []
		# randomly sample indexes (without replacement) from the
		# input data
    idxs = np.random.choice(range(0, len(images)), size=total, replace=False)
  	# loop over the indexes 
    for i in idxs:
      # # grab the current image and label
      image = images[i]
      label = labels[i]

      # chuyen ve thang [0; 255] de tao nhieu
      # image =  np.clip(image, 0, 255).astype('uint8')

      # generate an adversarial 
      baseImage = numpy.expand_dims(image, axis=0)
      baseImage = numpy.array(baseImage).astype(numpy.float32)
      baseImage = tf.constant(baseImage, dtype=tf.float32)

      delta = tf.Variable(tf.zeros_like(baseImage, dtype = 'float32'), trainable=True) 
      print("[INFO] generating perturbation...")
      deltaUpdated = generate_adversaries(model, baseImage, delta, label, steps = 100)

      print("[INFO] creating adversarial example...")
      adverImage = (baseImage + deltaUpdated).numpy().squeeze()
      # adverImage = np.clip(adverImage, 0, 255).astype('uint8')

      # preprocess de cho vao mo hinh train
      adverImage = tf.keras.applications.mobilenet_v2.preprocess_input(adverImage)
      
      # update our perturbed images and labels lists
      perturbImages.append(adverImage)
      perturbLabels.append(label)

    # yield the perturbed images and labels
    yield (np.array(perturbImages), np.array(perturbLabels))

In [ ]:
X_test2, Y_test2 = prep_dataset(X_test,Y_test)
(loss, acc) = model.evaluate(x=X_test2, y=Y_test2, verbose=0)
print("[INFO] normal testing images:")
print("[INFO] loss: {:.4f}, acc: {:.4f}\n".format(loss, acc))

[INFO] normal testing images:
[INFO] loss: 0.0275, acc: 0.9929



In [ ]:
# generate a set of adversarial from our test set
print("[INFO] generating adversarial examples with PGD...\n")
# (advX, advY) = next(generate_adversarial_batch(model, len(X_test),
# 	X_test, Y_test))

(advX, advY) = next(generate_adversarial_batch(model, len(X_test),
	X_test, Y_test))

advY = to_categorical(np.array(advY))

# re-evaluate the model on the adversarial images
(loss, acc) = model.evaluate(x=advX, y=advY, verbose=0)
print("[INFO] adversarial testing images:")
print("[INFO] loss: {:.4f}, acc: {:.4f}\n".format(loss, acc))

Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
step: 90, loss: -12.835275650024414...
step: 95, loss: -24.3377742767334...
[INFO] creating adversarial example...
[INFO] generating perturbation...
step: 0, loss: -5.960462772236497e-07...
step: 5, loss: -1.5497195136049413e-06...
step: 10, loss: -0.00033468366018496454...
step: 15, loss: -10.618736267089844...
step: 20, loss: -17.679935455322266...
step: 25, loss: -23.918195724487305...
step: 30, loss: -29.017505645751953...
step: 35, loss: -32.95988464355469...
step: 40, loss: -36.58026123046875...
step: 45, loss: -41.04997253417969...
step: 50, loss: -45.99574279785156...
step: 55, loss: -51.145790100097656...
step: 60, loss: -57.029701232910156...
step: 65, loss: -64.68096923828125...
step: 70, loss: -69.98483276367188...
step: 75, loss: -74.14954376220703...
step: 80, loss: -77.24268341064453...
step: 85, loss: -80.17996215820312...
step: 90, loss: -82.73355102539062...
step: 95, loss: -84.94630432128906...
[INFO] creating 

In [ ]:
# fine-tune our CNN on the adversarial images
print("[INFO] fine-tuning network on adversarial examples...")
model.fit(advX, advY,
	batch_size=64,
	epochs=5,
	validation_data=(X_test2,Y_test2),
	verbose=1)

[INFO] fine-tuning network on adversarial examples...
Epoch 1/5
29/29 [==============================] - 3s 67ms/step - loss: 4.0876 - accuracy: 0.4143 - val_loss: 3.3912 - val_accuracy: 0.3103
Epoch 2/5
29/29 [==============================] - 1s 38ms/step - loss: 1.1807 - accuracy: 0.5988 - val_loss: 1.7998 - val_accuracy: 0.3870
Epoch 3/5
29/29 [==============================] - 1s 38ms/step - loss: 0.8154 - accuracy: 0.7343 - val_loss: 1.1366 - val_accuracy: 0.6434
Epoch 4/5
29/29 [==============================] - 1s 33ms/step - loss: 0.6307 - accuracy: 0.7817 - val_loss: 0.7945 - val_accuracy: 0.7071
Epoch 5/5
29/29 [==============================] - 1s 33ms/step - loss: 0.4774 - accuracy: 0.8378 - val_loss: 0.7152 - val_accuracy: 0.7599


In [ ]:
# Sau khi train thêm vài epoch
# now that our model is fine-tuned we should evaluate it on the test
# set (i.e., non-adversarial) again to see if performance has degraded
(loss, acc) = model.evaluate(x=X_test2, y=Y_test2, verbose=0)
print("")
print("[INFO] normal testing images *after* fine-tuning:")
print("[INFO] loss: {:.4f}, acc: {:.4f}\n".format(loss, acc))

# do a final evaluation of the model on the adversarial images
(loss, acc) = model.evaluate(x=advX, y=advY, verbose=0)
print("[INFO] adversarial images *after* fine-tuning:")
print("[INFO] loss: {:.4f}, acc: {:.4f}".format(loss, acc))


[INFO] normal testing images *after* fine-tuning:
[INFO] loss: 0.0229, acc: 0.9935

[INFO] adversarial images *after* fine-tuning:
[INFO] loss: 0.5680, acc: 0.8873


In [ ]:
model.save('saved_new_model/my_adverTraining_model.h5')